### Introdução

Vamos criar um modelo de Machine Learning para prever as mortes num acidente, 
mas você precisa tomar muito cuidado com um dos tipos de erro.
Como abordar um problema desse tipo, onde um dos erros é muito mais custoso que o outro?

### Carrega Bibliotecas

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

%load_ext nb_black

<IPython.core.display.Javascript object>

### Carrega Dataset

In [2]:
titanic = pd.read_csv('train.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<IPython.core.display.Javascript object>

In [3]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


<IPython.core.display.Javascript object>

Algumas variáveis são irrelevantes para o modelo, não possuem poder preditivo algum.

In [4]:
titanic = titanic.drop(
    columns = ['Cabin', 'Ticket', 'PassengerId', 'Name']
)

<IPython.core.display.Javascript object>

In [5]:
titanic.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

<IPython.core.display.Javascript object>

In [6]:
titanic.Sex = titanic.Sex.map({'male':0, 'female':1})

<IPython.core.display.Javascript object>

In [7]:
titanic.isna().mean()

Survived    0.000000
Pclass      0.000000
Sex         0.000000
Age         0.198653
SibSp       0.000000
Parch       0.000000
Fare        0.000000
Embarked    0.002245
dtype: float64

<IPython.core.display.Javascript object>

In [8]:
titanic.Embarked.value_counts(dropna=False)

S      644
C      168
Q       77
NaN      2
Name: Embarked, dtype: int64

<IPython.core.display.Javascript object>

In [9]:
titanic.Embarked.value_counts(1)

S    0.724409
C    0.188976
Q    0.086614
Name: Embarked, dtype: float64

<IPython.core.display.Javascript object>

Ter apenas duas observacoes excluidas nao afetaria nosso modelo de forma relevante. Entretanto, se isso aconteceu no treino, o que garante que nao vai acontecer em producao? Devemos deixar o algoritmo pronto para lidar com isso. Como é um problema que parece ser muito pouco frequente, sendo quase inexistente, vamos simplesmente preencher o campo com a classe mais frequente(no aso, a classe 'S').

### Modelo de Machine Learning

In [10]:
titanic.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

<IPython.core.display.Javascript object>

In [11]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
target = 'Survived'

<IPython.core.display.Javascript object>

In [12]:
titanic.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,S
1,1,1,1,38.0,1,0,71.2833,C
2,1,3,1,26.0,0,0,7.9250,S
3,1,1,1,35.0,1,0,53.1000,S
4,0,3,0,35.0,0,0,8.0500,S


<IPython.core.display.Javascript object>

In [13]:
X = titanic[features]
y = titanic[target]

<IPython.core.display.Javascript object>

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

<IPython.core.display.Javascript object>

In [15]:
X_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
709,3,0,NaN,1,1,15.2458,C
439,2,0,31.0,0,0,10.5000,S
840,3,0,20.0,0,0,7.9250,S
720,2,1,6.0,0,1,33.0000,S
39,3,1,14.0,1,0,11.2417,C
...,...,...,...,...,...,...,...
433,3,0,17.0,0,0,7.1250,S
773,3,0,NaN,0,0,7.2250,C
25,3,1,38.0,1,5,31.3875,S
84,2,1,17.0,0,0,10.5000,S


<IPython.core.display.Javascript object>

In [16]:
categorical_features = ['Pclass', 'SibSp', 'Parch', 'Embarked']
numerical_features = ['Age', 'Sex', 'Fare']

<IPython.core.display.Javascript object>

In [17]:
categorical_pipe = Pipeline(
    [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', ce.OneHotEncoder()),
    ]
)

numerical_pipe = Pipeline(
    [
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
    ]
)

transformer = ColumnTransformer(
    [
        ('categorical_transformer', categorical_pipe, categorical_features),
        ('numerical_transformer', numerical_pipe, numerical_features),
    ]
)

X_train_transformed = transformer.fit_transform(X_train)
X_test_transformed = transformer.transform(X_test)

<IPython.core.display.Javascript object>

In [18]:
logit = LogisticRegression()

<IPython.core.display.Javascript object>

In [19]:
logit.fit(X_train_transformed, y_train)

LogisticRegression()

<IPython.core.display.Javascript object>

In [20]:
y_pred = logit.predict(X_test_transformed)

<IPython.core.display.Javascript object>

In [21]:
y_pred

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1])

<IPython.core.display.Javascript object>

In [22]:
y_test.values

array([1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1])

<IPython.core.display.Javascript object>

In [23]:
from sklearn.metrics import(
    accuracy_score,
    roc_auc_score,
    f1_score,
    precision_score,
    recall_score,
)

print(f"Acurácia: {accuracy_score(y_test, y_pred):.2f}")
print(f"Recall: {recall_score(y_test, y_pred):.2f}")
print(f"Precision: {precision_score(y_test, y_pred):.2f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.2f}")
print(f"ROC/AUC: {roc_auc_score(y_test, y_pred):.2f}")

Acurácia: 0.79
Recall: 0.70
Precision: 0.78
F1-Score: 0.74
ROC/AUC: 0.78


<IPython.core.display.Javascript object>

Vamos lembrar duas dessas métricas:

Precision = TP/(TP + FP)

Recall = TP/(TP+FN)

### E se o risco de errar uma predição for muito alto?

Numa situação em que não há mais espaço para melhorar o modelo e que a necessidade de evitar o erro tipo 1, ou o erro tipo 2 for muito alta, uma possibilidade é voce trabalhar o threshold da predicao.

O threshold nada mais é do que a probabilidade que o scikit-learn usa para determinar se a predição é 0 ou 1. Por default, o sklearn vai dizer que a predição é 1 quando a probabilidade de ser 1 for maior que 50%. Mas é possivel alterar esse valor!

Por exemplo, no nosso caso, é bem pior errar dizendo que alguem nao vai morrer e essa pessoa acabar morrendo, porque a ideia do nosso modelo é impedir mortes. Veja que se o modelo falar que a pessoa vai morrer e ela não morre, a perda não é tão grave. Portanto, vamos reduzir ao máximo o erro tipo 2.

In [24]:
# Predicao usando threshold como 50%:

logit.predict(X_test_transformed)

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1])

<IPython.core.display.Javascript object>

Ao inves de pegar a predicao diretamente, podemos pegar a probabilidade dos eventos:

In [25]:
# Pegando a probabilidade de ser 0 e 1.
logit.predict_proba(X_train_transformed)

array([[0.73168844, 0.26831156],
       [0.73253566, 0.26746434],
       [0.91713911, 0.08286089],
       ...,
       [0.94600952, 0.05399048],
       [0.07522482, 0.92477518],
       [0.45584976, 0.54415024]])

<IPython.core.display.Javascript object>

Veja que o primeiro individuo tem probabilidade de 73% de 0 (nao morrer) e de 26% de 1 (de morrer).Como o Scikit-Learn usa um threshold de 50%, ele vai dizer que o primeiro individuo nao vai morrer, pois ele tem probabilidade acima de 50% de nao morrer.

É com esse threshold que podemos brincar. O Scikit-Learn considera 50% de chance de morrer o valor minimo para considerar que o individuo vai morrer. Como nós queremos evitar todo o tipo de morte, poderiamos diminuir esse valor. Afinal, se considerarmos que 30% de chance de morte já é um risco alto para uma pessoa morrer, precisamos entao que o threshold nao seja 50%, mais 30%. E é isso que eu quero que voce aprenda hoje!

Para alterar o threshold que voce vai usar é bem simples, veja só:

In [26]:
# Para pegar a probabilidade de ser 1, que é a que é usada plo sklearn
# logit.predict_proba(X_test_transformed)[:, 1]

<IPython.core.display.Javascript object>

In [33]:
y_pred_novo = (logit.predict_proba(X_test_transformed)[:, 1] >= 0.15).astype('bool')
y_pred_novo

array([ True,  True, False,  True,  True,  True,  True, False,  True,
        True,  True, False,  True, False,  True,  True,  True,  True,
        True,  True, False,  True,  True, False, False, False,  True,
        True, False,  True, False,  True,  True,  True, False, False,
        True,  True,  True, False,  True, False, False, False,  True,
        True, False, False, False,  True,  True,  True, False,  True,
       False,  True,  True,  True,  True,  True, False,  True,  True,
        True, False,  True,  True, False,  True,  True,  True,  True,
        True,  True, False, False,  True,  True,  True,  True, False,
        True,  True, False,  True,  True,  True,  True,  True, False,
       False,  True,  True, False, False, False,  True, False,  True,
       False,  True, False,  True, False, False, False,  True,  True,
       False,  True,  True, False,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,

<IPython.core.display.Javascript object>

In [34]:
from sklearn.metrics import(
    accuracy_score,
    roc_auc_score,
    f1_score,
    precision_score,
    recall_score,
)

print(f"Acurácia: {accuracy_score(y_test, y_pred_novo):.2f}")
print(f"Recall: {recall_score(y_test, y_pred_novo):.2f}")
print(f"Precision: {precision_score(y_test, y_pred_novo):.2f}")
print(f"F1-Score: {f1_score(y_test, y_pred_novo):.2f}")
print(f"ROC/AUC: {roc_auc_score(y_test, y_pred_novo):.2f}")

Acurácia: 0.70
Recall: 0.93
Precision: 0.58
F1-Score: 0.72
ROC/AUC: 0.73


<IPython.core.display.Javascript object>